In [95]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 300)
import pickle

## Step 1: Read in hold out data, scalers, and best model

In [96]:
df = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [97]:
df

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,263000018,20140521T000000,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
4319,6600060120,20150223T000000,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
4320,1523300141,20140623T000000,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
4321,291310100,20150116T000000,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [98]:
print(len(model.coef_))

122


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4322 entries, 0 to 4322
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4322 non-null   int64  
 1   date           4322 non-null   object 
 2   bedrooms       4322 non-null   int64  
 3   bathrooms      4322 non-null   float64
 4   sqft_living    4322 non-null   int64  
 5   sqft_lot       4322 non-null   int64  
 6   floors         4322 non-null   float64
 7   waterfront     4322 non-null   int64  
 8   view           4322 non-null   int64  
 9   condition      4322 non-null   int64  
 10  grade          4322 non-null   int64  
 11  sqft_above     4322 non-null   int64  
 12  sqft_basement  4322 non-null   int64  
 13  yr_built       4322 non-null   int64  
 14  yr_renovated   4322 non-null   int64  
 15  zipcode        4322 non-null   int64  
 16  lat            4322 non-null   float64
 17  long           4322 non-null   float64
 18  sqft_liv

In [100]:
df.isna().sum()
df.drop(columns= ['date', 'id'], inplace=True)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [101]:
#create categories for bathrooms
df['bathrooms_r']= df['bathrooms'] *100
df['bathrooms_r'] = df['bathrooms_r'].astype(int)
conditions= [
     df['bathrooms_r'].isin(range(0,300)),
     df['bathrooms_r'].isin(range(300,500)),
     df['bathrooms_r'].isin(range(500,760)),
     df['bathrooms_r'].isin(range(760,900))
     ]
choices= [
    '0-2',
    '3-4',
    '5-6',
    '7-8'
]
df['bathrooms_r']= np.select(conditions, choices, default= 9)

In [102]:
df['bed_x_bath']= df['bathrooms']*df['bedrooms']
#capping sqft_lot at 200,000 - a bit more than 3 standard deviations away from the mean 
df['sqft_lot_cap']= np.clip(df['sqft_lot'], a_max=200000, a_min=None)
#Engineering a new feature to estimate size of outdoor space-- yard
df['yard']= abs(df['sqft_lot'] - (df['sqft_living']/df['floors']))
#Squaring sqftliving
df['sqft_living^2'] = df['sqft_living']**2
#waterview
df['water_x_view']= df['waterfront'] *df['view']
#sqft x condition
df['sqft_condition']= df['sqft_living']*df['condition']

conditions= [
    (df['condition']==1),
    (df['condition']==2),
    (df['condition']==3),
    (df['condition']==4),
    (df['condition']==5)
]
choices= [
    'Poor',
    'Poor',
    'Average',
    'Average',
    'High']

df['conditions_2']= np.select(conditions, choices, default= df['condition'])

conditions= [
    df['grade'].isin(range(1,7)),
    df['grade'].isin(range(7,8)),
    df['grade'].isin(range(8,9)),
    df['grade'].isin(range(9,14))
]

choices= [
    'Poor',
    'Average',
    'Good',
    'High']

df['grade_cat']= np.select(conditions, choices)

df['cond_x_grad']= df['condition']*df['grade']
df['sqft_condition']= df['sqft_living']*df['condition']

#Created a new column of whether or not there is a basement
df['basement']= np.where(df['sqft_basement'].values > 0, 1, 0)
#created year since reno
conditions= [
    (df['yr_renovated']!=0), 
    (df['yr_renovated']==0),
]
choices= [
    (2020- df['yr_renovated']),
    0   
]
#renovation
df['yrs_post_reno']= np.select(conditions, choices, 99)
#diving by latitude dummy
df['lat'].min()+ ((df['lat'].max() - df['lat'].min())/2)
df['North'] = np.where(df['lat']>47.5, 1, 0)
df['South'] = np.where(df['lat']<47.5, 1, 0)
#diving by latitude category
df['North_South']= np.where(df['lat']>47.5, 'North', 'South')


ENGINEERING DUMMIES

In [103]:
bath_dum= pd.get_dummies(df, columns=['bathrooms_r'], drop_first=True)
NorthSouth_dum= pd.get_dummies(df, columns=['North_South'])
conditions2_dum= pd.get_dummies(df, columns= ['conditions_2'], drop_first=True)
view_dum= pd.get_dummies(df, columns= ['view'], drop_first=True)
grade_cat_dum= pd.get_dummies(df, columns=['grade_cat'], drop_first=True)
zipcode_dum= pd.get_dummies(df, columns=['zipcode'])


In [104]:
df_alldum= pd.get_dummies(df, columns= ['bathrooms_r','grade_cat', 'conditions_2', 'zipcode'])

In [105]:
df9 = df_alldum.drop(columns=['grade','North_South', 'water_x_view',])


In [106]:
df9.columns.tolist()

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'bed_x_bath',
 'sqft_lot_cap',
 'yard',
 'sqft_living^2',
 'sqft_condition',
 'cond_x_grad',
 'basement',
 'yrs_post_reno',
 'North',
 'South',
 'bathrooms_r_0-2',
 'bathrooms_r_3-4',
 'bathrooms_r_5-6',
 'grade_cat_Average',
 'grade_cat_Good',
 'grade_cat_High',
 'grade_cat_Poor',
 'conditions_2_Average',
 'conditions_2_High',
 'conditions_2_Poor',
 'zipcode_98001',
 'zipcode_98002',
 'zipcode_98003',
 'zipcode_98004',
 'zipcode_98005',
 'zipcode_98006',
 'zipcode_98007',
 'zipcode_98008',
 'zipcode_98010',
 'zipcode_98011',
 'zipcode_98014',
 'zipcode_98019',
 'zipcode_98022',
 'zipcode_98023',
 'zipcode_98024',
 'zipcode_98027',
 'zipcode_98028',
 'zipcode_98029',
 'zipcode_98030',
 'zipcode_98031',
 'zipcode_98032',
 'zipcode_98033',
 'zipcode_98034',
 'zipcode_98038

In [107]:
len(df9.columns)

106

In [108]:
#save df to csv
df.to_csv('data/df_holdout')

In [109]:
#transformed_holdout = final_scaler(holdout)

## Step 3: Predict the holdout set

In [120]:
infile = open("model.pickle", 'rb')
model = pickle.load(infile)
infile.close()

In [123]:
predictions = model.predict(df9)

In [110]:
# final_answers = final_model.predict(transformed_holdout)

In [113]:
#fit a model
from sklearn import linear_model

In [124]:
df_preds = pd.DataFrame(predictions)

## Step 4: Export your predictions

In [125]:
df_preds.to_csv('housing_preds_lera_tsayukova.csv')